In [30]:
import akshare as ak
import pandas as pd
import numpy as np
import requests
import functools

In [32]:
# 1. 一次性获取所有 A 股股票的实时行情数据
# 定义一个空的 DataFrame 用于存储所有数据
all_stock_spot = ak.stock_zh_a_spot_em()
if all_stock_spot.empty:
    print("Failed to get all stock spot data.")
else:
    # 将总市值从元转换为亿
    all_stock_spot['总市值'] = pd.to_numeric(all_stock_spot['总市值'], errors='coerce') / 1e8
    # 筛选出市值大于 300 亿的股票代码和名称
    valid_stocks = all_stock_spot[all_stock_spot['总市值'] > 100][['代码', '名称']].values.tolist()

In [33]:
# 2. 定义起始日期
start_date = '20000101'

# 3. 存储筛选后的股票数据
filtered_stocks_data = {}

# 4. 遍历市值符合要求的股票代码获取日线数据
for code, name in valid_stocks:
    try:
        # 获取股票日线数据
        stock_daily = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=start_date, adjust="qfq")
        if not stock_daily.empty:
            filtered_stocks_data[code] = {
                'name': name,
                'data': stock_daily
            }
    except Exception as e:
        print(f"Error getting daily data for {code}: {e}")

Error getting daily data for 000875: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [34]:
# 5. 定义计算 KDJ 的函数
def calculate_kdj(data, n=9, m1=3, m2=3):
    """
    计算 KDJ 指标
    :param data: 包含 'close', 'high', 'low' 列的 DataFrame
    :param n: RSV 周期，默认为 9
    :param m1: K 平滑因子，默认为 3
    :param m2: D 平滑因子，默认为 3
    :return: 包含 'K', 'D', 'J' 列的 DataFrame
    """
    low_min = data['low'].rolling(window=n).min()
    high_max = data['high'].rolling(window=n).max()
    rsv = (data['close'] - low_min) / (high_max - low_min) * 100
    data['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
    data['D'] = data['K'].ewm(com=m2 - 1, adjust=False).mean()
    data['J'] = 3 * data['K'] - 2 * data['D']
    return data

# 6. 计算筛选出的股票的 KDJ 值
kdj_results = {}
for code, stock_info in filtered_stocks_data.items():
    data = stock_info['data'].rename(columns={'开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low'})
    data_with_kdj = calculate_kdj(data)
    kdj_results[code] = {
        'name': stock_info['name'],
        'data': data_with_kdj
    }


In [35]:
# 7. 挑选出 KDJ 的 J 值小于 5 的股票
j_less_than_5_stocks = []
for code, stock_info in kdj_results.items():
    if not stock_info['data']['J'].empty and stock_info['data']['J'].iloc[-1] < -5 :
        j_less_than_5_stocks.append({
            'code': code,
            'name': stock_info['name'],
            'j_value': stock_info['data']['J'].iloc[-1]
        })

# 8. 打印 KDJ 的 J 值小于 5 的股票信息
if j_less_than_5_stocks:
    print("KDJ 的 J 值小于 5 的股票信息：")
    for stock in j_less_than_5_stocks:
        print(f"股票代码: {stock['code']}, 股票名称: {stock['name']}, J 值: {stock['j_value']:.2f}")
else:
    print("没有找到 KDJ 的 J 值小于 5 的股票。")

KDJ 的 J 值小于 5 的股票信息：
股票代码: 002217, 股票名称: *ST合泰, J 值: -6.36
股票代码: 600320, 股票名称: 振华重工, J 值: -5.22
股票代码: 600438, 股票名称: 通威股份, J 值: -5.42
股票代码: 601018, 股票名称: 宁波港, J 值: -11.96
股票代码: 601928, 股票名称: 凤凰传媒, J 值: -11.39
股票代码: 601872, 股票名称: 招商轮船, J 值: -8.41
股票代码: 601228, 股票名称: 广州港, J 值: -6.59
股票代码: 600925, 股票名称: 苏能股份, J 值: -6.73
股票代码: 600373, 股票名称: 中文传媒, J 值: -9.51
股票代码: 600170, 股票名称: 上海建工, J 值: -11.72
股票代码: 600028, 股票名称: 中国石化, J 值: -5.68
股票代码: 002739, 股票名称: 万达电影, J 值: -10.35
股票代码: 000630, 股票名称: 铜陵有色, J 值: -8.40
股票代码: 601958, 股票名称: 金钼股份, J 值: -6.43
股票代码: 600295, 股票名称: 鄂尔多斯, J 值: -5.97
股票代码: 000703, 股票名称: 恒逸石化, J 值: -14.53
股票代码: 600176, 股票名称: 中国巨石, J 值: -6.89
股票代码: 000060, 股票名称: 中金岭南, J 值: -8.19
股票代码: 601666, 股票名称: 平煤股份, J 值: -7.06
股票代码: 603993, 股票名称: 洛阳钼业, J 值: -9.29
股票代码: 600755, 股票名称: 厦门国贸, J 值: -8.33
股票代码: 600820, 股票名称: 隧道股份, J 值: -5.83
股票代码: 600098, 股票名称: 广州发展, J 值: -7.15
股票代码: 601016, 股票名称: 节能风电, J 值: -6.01
股票代码: 600362, 股票名称: 江西铜业, J 值: -6.02
股票代码: 000937, 股票名称: 冀中能源, J 值: -6.63
股票代码: 600325,

In [51]:
# 7. 收集 J 值小于 5 时第二天的涨跌幅度
next_day_returns = []
for code, stock_info in kdj_results.items():
    data = stock_info['data']
    j_less_than_5_indices = data[data['J'] < 0].index
    for idx in j_less_than_5_indices:
        next_idx = data.index.get_loc(idx) + 1
        if next_idx < len(data):
            current_close = data.loc[idx, 'close']
            next_close = data.loc[data.index[next_idx], 'close']
            return_rate = (next_close - current_close) / current_close * 100
            next_day_returns.append(return_rate)

# 8. 分析涨跌幅度分布
if next_day_returns:
    bins = [-np.inf, -5, -3, -1, 0, 1, 3, 5, np.inf]
    labels = ['<-5%', '-5%~-3%', '-3%~-1%', '-1%~0%', '0%~1%', '1%~3%', '3%~5%', '>5%']
    distribution = pd.cut(next_day_returns, bins=bins, labels=labels).value_counts()
    print("J 值小于 5 时第二天的涨跌幅度分布：")
    print(distribution)
else:
    print("没有找到 J 值小于 5 的情况，无法分析涨跌幅度分布。")

C:\Users\应益龙\AppData\Local\Temp\ipykernel_29708\2566744693.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  return_rate = (next_close - current_close) / current_close * 100
C:\Users\应益龙\AppData\Local\Temp\ipykernel_29708\2566744693.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return_rate = (next_close - current_close) / current_close * 100


J 值小于 5 时第二天的涨跌幅度分布：
<-5%       11861
-5%~-3%     8305
-3%~-1%    21408
-1%~0%     24373
0%~1%      21758
1%~3%      25770
3%~5%      10194
>5%        11978
Name: count, dtype: int64


In [25]:
# 4. 遍历市值符合要求的股票代码获取周线数据
for code, name in valid_stocks:
    try:
        # 获取股票周线数据，将 period 参数改为 'weekly'
        stock_weekly = ak.stock_zh_a_hist(symbol=code, period="weekly", start_date=start_date, adjust="qfq")
        if not stock_weekly.empty:
            filtered_stocks_data[code] = {
                'name': name,
                'data': stock_weekly
            }
    except Exception as e:
        print(f"Error getting weekly data for {code}: {e}")

In [26]:
# 5. 定义计算 KDJ 的函数
def calculate_kdj(data, n=9, m1=3, m2=3):
    """
    计算 KDJ 指标
    :param data: 包含 'close', 'high', 'low' 列的 DataFrame
    :param n: RSV 周期，默认为 9
    :param m1: K 平滑因子，默认为 3
    :param m2: D 平滑因子，默认为 3
    :return: 包含 'K', 'D', 'J' 列的 DataFrame
    """
    low_min = data['low'].rolling(window=n).min()
    high_max = data['high'].rolling(window=n).max()
    rsv = (data['close'] - low_min) / (high_max - low_min) * 100
    data['K'] = rsv.ewm(com=m1 - 1, adjust=False).mean()
    data['D'] = data['K'].ewm(com=m2 - 1, adjust=False).mean()
    data['J'] = 3 * data['K'] - 2 * data['D']
    return data

# 6. 计算筛选出的股票的 KDJ 值
kdj_results_weekly = {}
for code, stock_info in filtered_stocks_data.items():
    data = stock_info['data'].rename(columns={'开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low'})
    data_with_kdj = calculate_kdj(data)
    kdj_results_weekly[code] = {
        'name': stock_info['name'],
        'data': data_with_kdj
    }


In [28]:
# 7. 挑选出 KDJ 的 J 值小于 5 的股票
j_less_than_5_stocks = []
for code, stock_info in kdj_results_weekly.items():
    if not stock_info['data']['J'].empty and stock_info['data']['J'].iloc[-1] < 0:
        j_less_than_5_stocks.append({
            'code': code,
            'name': stock_info['name'],
            'j_value': stock_info['data']['J'].iloc[-1]
        })

# 8. 打印 KDJ 的 J 值小于 5 的股票信息
if j_less_than_5_stocks:
    print("KDJ 的 J 值小于 5 的股票信息（周线数据）：")
    for stock in j_less_than_5_stocks:
        print(f"股票代码: {stock['code']}, 股票名称: {stock['name']}, J 值: {stock['j_value']:.2f}")
else:
    print("没有找到 KDJ 的 J 值小于 5 的股票（周线数据）。")

KDJ 的 J 值小于 5 的股票信息（周线数据）：
股票代码: 601933, 股票名称: 永辉超市, J 值: -8.01
股票代码: 605499, 股票名称: 东鹏饮料, J 值: -6.88
股票代码: 601857, 股票名称: 中国石油, J 值: -11.98
股票代码: 600236, 股票名称: 桂冠电力, J 值: -2.68
股票代码: 601088, 股票名称: 中国神华, J 值: -7.18
股票代码: 600938, 股票名称: 中国海油, J 值: -12.76
股票代码: 601111, 股票名称: 中国国航, J 值: -6.31
股票代码: 603338, 股票名称: 浙江鼎力, J 值: -4.64
股票代码: 600028, 股票名称: 中国石化, J 值: -3.47
股票代码: 000729, 股票名称: 燕京啤酒, J 值: -5.08
股票代码: 000333, 股票名称: 美的集团, J 值: -4.17
股票代码: 600029, 股票名称: 南方航空, J 值: -8.46
股票代码: 600600, 股票名称: 青岛啤酒, J 值: -4.16
股票代码: 002966, 股票名称: 苏州银行, J 值: -16.25
股票代码: 600015, 股票名称: 华夏银行, J 值: -5.99
股票代码: 601816, 股票名称: 京沪高铁, J 值: -4.62
股票代码: 300979, 股票名称: 华利集团, J 值: -12.90
股票代码: 600968, 股票名称: 海油发展, J 值: -4.39
股票代码: 601021, 股票名称: 春秋航空, J 值: -0.70
股票代码: 600079, 股票名称: 人福医药, J 值: -7.61
股票代码: 002203, 股票名称: 海亮股份, J 值: -1.04
股票代码: 603156, 股票名称: 养元饮品, J 值: -4.90
股票代码: 300888, 股票名称: 稳健医疗, J 值: -5.32
股票代码: 600575, 股票名称: 淮河能源, J 值: -8.92
股票代码: 600548, 股票名称: 深高速, J 值: -11.18
股票代码: 600655, 股票名称: 豫园股份, J 值: -3.10
股票代码: 6

In [29]:
stock_df = ak.stock_zh_a_spot_em()
